# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at mlta-2022-spring@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1H5ZONrb2LMOCixLY7D5_5-7LkIaXO6AGEaV2mRdTOMY/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/c/ml2022spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2.5hrs
  

## Download Dataset

In [1]:
!pip install gdown

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
# Download link 1
!gdown --id '1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1qwjbRjq481lHsnTrrF4OjKQnxzgoLEFR' --output hw7_data.zip

# Download Link 3 (if the above link fails) 
# !gdown --id '1QXuWjNRZH6DscSd6QcRER0cnxmpZvijn' --output hw7_data.zip

!unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

/home/weiweichi/.local/lib/python3.8/site-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb
To: /home/weiweichi/weichi/HW7/hw7_data.zip
100%|███████████████████████████████████████| 9.57M/9.57M [00:00<00:00, 108MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          
Fri May  6 19:28:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Comput

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [3]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


## Import Packages

In [4]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast, get_linear_schedule_with_warmup, AutoModelForQuestionAnswering, BertTokenizer

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.cuda.empty_cache()
# Fix random seed for reproducibility
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(10942178)

In [5]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = False

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

# Load Model and Tokenizer




 

In [6]:
# model = BertForQuestionAnswering.from_pretrained('hfl/chinese-pert-large').to(device)
tokenizer = BertTokenizerFast.from_pretrained('hfl/chinese-pert-large')
# model = AutoModelForQuestionAnswering.from_pretrained('wptoux/albert-chinese-large-qa').to(device)
# tokenizer = BertTokenizer.from_pretrained('wptoux/albert-chinese-large-qa')
# https://huggingface.co/uer/roberta-base-chinese-extractive-qa
# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

## Read Data

- Training set: 31690 QA pairs
- Dev set: 4131  QA pairs
- Test set: 4957  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [7]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("./hw7_train.json")
dev_questions, dev_paragraphs = read_data("./hw7_dev.json")
test_questions, test_paragraphs = read_data("./hw7_test.json")

## Tokenize Data

In [8]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 
tokenizer.tokenize('↑', add_special_tokens=False)
tokenizer.tokenize('↓', add_special_tokens=False)
tokenizer.tokenize('←', add_special_tokens=False)
tokenizer.tokenize('→', add_special_tokens=False)
tokenizer.tokenize('※', add_special_tokens=False)
dev_paragraphs = [i.replace(' ','※').replace('\u200b','↑').replace('\u200e', '↓').replace('\u3000', '←').replace('#','→') for i in dev_paragraphs]
test_paragraphs = [i.replace(' ','※').replace('\u200b','↑').replace('\u200e', '↓').replace('\u3000', '←').replace('#','→') for i in test_paragraphs]

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

In [9]:
for i, par in enumerate(dev_paragraphs):
    if '↑' in par or '↓' in par or '←' in par or '→' in par:
        print(i, "\n", par)
        break

706 
 阿爾泰共和國，羅馬化：Respublika※Altay；是俄羅斯聯邦主體之一，屬西伯利亞聯邦管區。面積92,600平方公里，人口202,947。首府戈爾諾-阿爾泰斯克。原名戈爾諾-阿爾泰自治州。1992年改為今名。農業是阿爾泰共和國的主要經濟部門，農產品占共和國年社會總產品的65%，占全俄羅斯年農產品總量的0.3%，重要部門是畜牧業和養蜂業。種植業也是阿爾泰農業的主要部門之一，不過它在農業中的比重明顯低於畜牧業。當地主要農作物有穀物、馬鈴薯、胡蘿蔔等蔬菜以及經濟作物。※←←養蜂業：阿爾泰人利用山區自然條件養蜂已數百年歷史。今天養蜂業仍然是該共和國最主要的農業活動之一，蜂蜜產量每年高達數千噸。當地養蜂業的優勢是蜂蜜品質上乘，明顯優於俄羅斯其他地區的同類產品。食品工業是阿爾泰共和國工業的主要部門之一，境內有數家大型食品加工企業。主要生產麵包製品、無酒精飲料、香腸、奶油、奶酪、蔬菜水果罐頭和葡萄酒．大部分產品滿足共和國內市場需求，一小部分供給毗鄰地區。


# Dataset and Dataloader

In [10]:
doc_stride = 150
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 350
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = doc_stride

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = int((answer_start_token + answer_end_token) // (random.uniform(1, 4)))
            # mid = int(random.uniform(answer_start_token, answer_end_token))
            paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 4 

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

# Function for Evaluation

In [11]:
def index_before_tokenize(tokens, paragraph, start, end):
    char_count, new_start, new_end = 0, 512, 512
        
    for i, token in enumerate(tokens):
        if i == start:
            new_start = char_count
        if token == '[UNK]' or token == '[CLS]' or token == '[SEP]':
            if token != '[UNK]':
                char_count += 1
            else:
                while char_count < len(paragraph) and paragraph[char_count].upper() != tokens[i+1][0].upper():
                    char_count += 1
                    if tokens[i+1] == '[UNK]':
                        break

        else:
            for c in token:
                char_count += c != '#'
        if i == end:
            new_end = char_count - 1
    return new_start, new_end

## Testing with fusion

In [12]:
def evaluate_fusion(i, data, output_list, doc_stride=150, paragraph=None, paragraph_tokenized=None):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    start_logits = output_list[0].start_logits
    end_logits = output_list[0].end_logits
    for output in output_list[1:]:
        start_logits += output.start_logits
        end_logits += output.end_logits
    start_logits = start_logits / len(output_list)
    end_logits = end_logits / len(output_list)
    answer1 = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    entire_start_index = 0
    entire_end_index = 0
    # print(paragraph, '\n', paragraph_tokenized)
    # print(paragraph_tokenized)
    for k in range(num_of_windows):
        
        # Obtain answer by choosing the most probable start position / end position
        mask = data[1][0][k].bool() & data[2][0][k].bool()
        mask = mask.to(device)
        masked_output_start = torch.masked_select(start_logits[k], mask)[:-1]
        start_prob, start_index = torch.max(masked_output_start, dim=0)
        masked_output_end = torch.masked_select(end_logits[k], mask)[start_index:-1]
        end_prob, end_index = torch.max(masked_output_end, dim=0)
        end_index += start_index
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        masked_data = torch.masked_select(data[0][0][k].cuda(), mask)[:-1]
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob and (end_index - start_index <= 25):# and (end_index > start_index):
            max_prob = prob
            entire_start_index = start_index.item() + doc_stride * k
            entire_end_index = end_index.item() + doc_stride * k
            
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer1 = tokenizer.decode(masked_data[start_index : end_index + 1])
            # Remove spaces (e.g "大 金" --> "大金") and replace
            # temp = answer1
            answer1 = answer1.replace(' ','').replace('※', ' ').replace('↑', '\u200b').replace('↓', '\u200e').replace('←', '\u3000').replace('→', '#')
    
    if "[UNK]" in answer1:
        new_start, new_end = index_before_tokenize(tokens=paragraph_tokenized, paragraph = paragraph, 
                                                   start=entire_start_index, end=entire_end_index)
        answer2 = paragraph[new_start: new_end+1]
        answer2 = answer2.replace('※', ' ').replace('↑', '\u200b').replace('↓', '\u200e').replace('←', '\u3000').replace('→', '#')
        # return answer2
    else:
        new_start, new_end = index_before_tokenize(tokens=paragraph_tokenized, paragraph = paragraph, 
                                                   start=entire_start_index, end=entire_end_index)
        answer2 = paragraph[new_start: new_end+1]
        answer2 = answer2.replace('※', ' ').replace('↑', '\u200b').replace('↓', '\u200e').replace('←', '\u3000').replace('→', '#')
        
    if answer1 != answer2:
        # print(temp)
        print(f"{i} | answer1 = {answer1} | answer2 = {answer2}")
        print("--------------------------")
        # raise TypeError
    return answer2 or answer1

In [13]:
model1 = BertForQuestionAnswering.from_pretrained("saved_model/type1").to(device) # luhua
model2 = BertForQuestionAnswering.from_pretrained("saved_model/type2").to(device) # hfl
model3 = BertForQuestionAnswering.from_pretrained("saved_model/type3").to(device) # uer

In [14]:
print("Evaluating Test Set ...")
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)
result = []
model1.eval()
model2.eval()
model3.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader):
        # if not (i == 3637 or i == 2180):continue
        # if not (i == 49 or i == 84 or i == 589 or i == 924 or i == 3166):continue
        output1 = model1(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output2 = model2(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output3 = model3(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate_fusion(i, data, [output1, output2, output3], doc_stride, test_paragraphs[test_questions[i]['paragraph_id']],
                               test_paragraphs_tokenized[test_questions[i]['paragraph_id']].tokens))
                               
result_file = "submission.csv"
with open(result_file, 'w') as f:
    f.write("ID,Answer\n")
    for i, test_question in enumerate(test_questions):
    # Replace commas in answers with empty strings (since csv is separated by comma)
    # Answers in kaggle are processed in the same way
        f.write(f"{test_question['id']},{result[i].replace(',','')}\n")
torch.cuda.empty_cache()
print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...
49 | answer1 = 大型購物中心megabox開幕 | answer2 = 大型購物中心MegaBox開幕
--------------------------
250 | answer1 = 溥[UNK] | answer2 = 溥儁
--------------------------
261 | answer1 = 人均gdp | answer2 = 人均GDP
--------------------------
332 | answer1 = 目前沒有觀察到任何語言純[UNK]以力道來區分不同輔音 | answer2 = 目前沒有觀察到任何語言純綷以力道來區分不同輔音
--------------------------
340 | answer1 = [UNK]人國 | answer2 = 荇人國
--------------------------
428 | answer1 = [UNK] | answer2 = 20㎝
--------------------------
563 | answer1 = 馬[UNK] | answer2 = 馬馼
--------------------------
589 | answer1 = ucla | answer2 = UCLA
--------------------------
635 | answer1 = 常[UNK] | answer2 = 常璩
--------------------------
698 | answer1 = dr. rufus b. von kleinsmid | answer2 = Dr. Rufus B. von Kleinsmid
--------------------------
938 | answer1 = [UNK]稻 | answer2 = 秈稻
--------------------------
991 | answer1 = 白[UNK]紀滅絕事件 | answer2 = 白堊紀滅絕事件
--------------------------
1154 | answer1 =  | answer2 = 兩
--------------------------
1196 | answer1 =